In [ ]:
%matplotlib inline

In [ ]:
import gc
import pathlib
import sys

In [ ]:
import joblib
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.inspection import permutation_importance

In [ ]:
src_dir = "../../src"

In [ ]:
sys.path.append(src_dir)

In [ ]:
from package.constants import *
from package.utils import *

In [ ]:
# hyperparameters
percentile = 90

In [ ]:
train = pd.read_parquet(train_path)

In [ ]:
# See https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/135896
is_train = (train["date"] >= train_start_date) & (train["date"] < validation_start_date)
is_valid = train["date"] >= validation_start_date

In [ ]:
dtrain = create_dataset(
    train,
    is_train,
    features,
    transformed_target,
    categorical_feature=categorical_features,
)
dvalid = create_dataset(
    train,
    is_valid,
    features,
    transformed_target,
    categorical_feature=categorical_features,
)

In [ ]:
del train
del is_train
del is_valid

In [ ]:
gc.collect()

In [ ]:
%%time
model = lgb.train(
    lgb_params,
    dtrain,
    early_stopping_rounds=100,
    valid_sets=[dtrain, dvalid],
    verbose_eval=10,
)

In [ ]:
ax = lgb.plot_importance(model, importance_type="split", figsize=(16, 9))

In [ ]:
ax = lgb.plot_importance(model, importance_type="gain", figsize=(16, 9))

In [ ]:
feature_importances = model.feature_importance(importance_type="gain")
threshold = np.percentile(feature_importances, 100 - percentile)
is_removed = feature_importances <= threshold
removed_features = np.array(features)
removed_features = removed_features[is_removed]
removed_features = list(removed_features)

In [ ]:
removed_features

In [ ]:
joblib.dump(removed_features, removed_features_path)